# Polygon-API Data Retrieval and Processing

#### Wrote sample functions to achieve the following goals:
1. Retrieve all ticker data from Polygon-API export into local csv files `get_tickers()`
2. Combine individual ticker data csv files into 1 ticker_data.csv `combine_tickers()`
3. Filter ticker_data.csv to include only relevant US exchanges as list `filter_us_exchange()`
4. Get bars (price data) for all relevant tickers from polygon-api as csv using `get_bars`
5. Get all split data from polygon-api as csv using `get_splits()`
6. Overwrite erroneous polygon_split.csv with manually made correct_splits.csv for relevant tickers using `fix_splits()`
7. Get all dividend data from polygon-api as csv using `get_divs()`
8. Combine all data into combine_bars.csv file for future adjustments  using `combine_bars()`
9. Fix all erroneous data using combined data frame and `adjust_bars()` function for proper price-split adjustments (essentially the main function)

### Check `Function Dictionary` category for detailed use of sample functions and their return values

##### Importing Libraries

In [2]:
# Importing Relevant Libraries

import requests
import pandas as pd
import matplotlib
import os
import math
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

###### Testing API Connection

In [4]:
# Testing API connection

api_key = "redacted"

tickers_url = f'https://api.polygon.io/v3/reference/tickers?active=true&sort=ticker&order=asc&limit=10&apiKey={api_key}'

requests.get(tickers_url)

<Response [200]>

### Sample Ticker Data Retrieval

In [28]:

# URL with API key
api_key = 'redacted'
POLYGON_TICKERS_URL = f'https://api.polygon.io/v3/reference/tickers?page={}&apiKey={api_key}'

# Sample limit for testing
limit = '10'

def get_tickers(POLYGON_TICKERS_URL, limit=False):
    page = 1
    
    # Make session
    session = requests.Session()
    
    # Ticker Count
    r = session.get(POLYGON_TICKERS_URL.format(page))
    data = r.json()
    print(data)
    
    # Figure out number of pages needed
    count = data['count']
    print('total tickers ' + str(count))
    pages = math.ceil(count / data['perPage'])
    
    # Pull in all the pages of tickers from Polygon.io
    
    # For Production
    if limit == False:
        for pages in range (2, pages+1):  # For production
            r = session.get(POLYGON_TICKERS_URL.format(page))
            data = r.json()
            df = pd.DataFrame(data['tickers'])
            df.to_csv('data/tickers/{}.csv'.format(page), index=False)
            print('Page {} processed'.format(page))
            page += 1
        
    # For Testing
    else:
        for pages in range (2, int(limit)):  # For testing
            r = session.get(POLYGON_TICKERS_URL.format(page))
            data = r.json()
            df = pd.DataFrame(data['tickers'])
            # Retrieve CSV of tickers
            df.to_csv('data/tickers/{}.csv'.format(page), index=False)
            print('Page {} processed'.format(page))
            page += 1
        
    
    # Return results
    return('Processes {} pages of tickers'.format(page-1))
print("done")

# Testing
get_tickers(POLYGON_TICKERS_URL, limit=10)

done
{'results': [{'ticker': 'A', 'name': 'Agilent Technologies Inc.', 'market': 'stocks', 'locale': 'us', 'primary_exchange': 'XNYS', 'type': 'CS', 'active': True, 'currency_name': 'usd', 'cik': '0001090872', 'composite_figi': 'BBG000BWQYZ5', 'share_class_figi': 'BBG001SCTQY4', 'last_updated_utc': '2021-06-30T00:00:00Z'}, {'ticker': 'AA', 'name': 'Alcoa Corporation', 'market': 'stocks', 'locale': 'us', 'primary_exchange': 'XNYS', 'type': 'CS', 'active': True, 'currency_name': 'usd', 'cik': '0001675149', 'composite_figi': 'BBG00B3T3HD3', 'share_class_figi': 'BBG00B3T3HF1', 'last_updated_utc': '2021-06-30T00:00:00Z'}, {'ticker': 'AAA', 'name': 'AAF First Priority CLO Bond ETF', 'market': 'stocks', 'locale': 'us', 'primary_exchange': 'ARCX', 'type': 'ETF', 'active': True, 'currency_name': 'usd', 'composite_figi': 'BBG00X5FSP48', 'share_class_figi': 'BBG00X5FSPZ4', 'last_updated_utc': '2021-06-30T00:00:00Z'}, {'ticker': 'AAAU', 'name': 'Goldman Sachs Physical Gold ETF Shares', 'market': '

KeyError: 'perPage'

# Function Dictionary:

## Creating Tickers Dataframe, Filtering Results and Correcting Erroneous Data:

- Directory of functions for data retrieval, filtering and correction
- Eventually hoping to put constant API variables into separate config file

##### `get_tickers(url = POLYGON_TICKERS_URL)` : gets all available ticker data
> ##### Arguments: 
- `POLYGON_TICKERS_URL` : Polygon-API key with link to tickers data 

> ##### Returns:
- All available polygon tickers data as CSV file

##### `combine_tickers(directory)` : combines ticker data all into singular csv file indexed by ticker

> ##### Arguments: 
- Directory : directory containing `get_tickers()` csv-data

> ##### Returns:
- CSV file of ticker data from `get_tickers()` indexed by ticker

##### `filter_us_exchange(ticker_df)` : filters tickers to only return tickers on relevant US exchanges

> ##### Arguments: 
- ticker_df : DF of CSV file from `combine_tickers()`

> ##### Returns:
- `symbolslist` : List of ticker symbols on relevant US exchanges

##### `get_bars(symbolslist, outdir, start, end)` : gets price data as bars

> ##### Arguments: 
- `symbolslist` : List of ticker symbols on relevant US exchanges
- `outdir` : Output directory
- `start` : starting date
- `end` : ending date

> ##### Returns:
- CSV of Price data (formatted as bars (open, close, high, low, volume)) for each ticker

##### `get_splits(symbolslist, outdir)` : gets polygon split data

> ##### Arguments: 
- `symbolslist` : List of ticker symbols on relevant US exchanges
- `outdir` : Output directory

> ##### Returns:
- All available polygon split data for split data as CSV

##### `fix_splits(splitpath)` : adjusts erroneous polygon split data for corrected split

> ##### Arguments: 
- `split_corrections.csv` : manually created DF of correct split data
- `splitpath` : Path to Polygon-API split data csv file

> ##### Returns:
- Overwrites splitpath for corrected data if necessary

##### `get_divs(symbolslist, outdir)` : gets polygon dividend data

> ##### Arguments: 
- `symbolslist` : List of ticker symbols on relevant US exchanges
- `outdir` : Output directory


> ##### Returns:
- All available polygon tickers dividend data

##### `combine_bars(barpath, splitpath, divpath)`  : Secondary "Main" function combining all data with correct / incorrect split ratio data

> ##### Arguments: 
- `barpath` : bar price data csv path
- `splitpath` : corrected splitpath csv path
- `divpath` : divident data csv path

> ##### Returns:
- csv file of all combined data

##### `adj_bars(directory)` : Essentially the "Main" function adjusts incorrect data with corrected split ratio

> ##### Arguments: 
- `directory` : path to cvs file of all correct data from `combine_bars`

> ##### Returns:
- csv file of all data adjusted for split ratio

In [ ]:
import requests
import pandas as pd
import matplotlib
import os
import math
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


#%%
# Set some constant variables, I could put all of this in a seperate config file
ALPACA_API_KEY = os.environ.get('ALPACA_API_KEY')
START = '2005-01-03'
END = '2020-10-23'
# URL for all the tickers on Polygon
POLYGON_TICKERS_URL = 'https://api.polygon.io/v2/reference/tickers?page={}&apiKey={}'
# URL FOR PRICING DATA - Note, getting pricing that is UNADJUSTED for splits, I will try and adjust those manually
POLYGON_AGGS_URL = 'https://api.polygon.io/v2/aggs/ticker/{}/range/1/day/{}/{}?unadjusted=true&apiKey={}'
# URL FOR DIVIDEND DATA
POLYGON_DIV_URL = 'https://api.polygon.io/v2/reference/dividends/{}?apiKey={}'
# URL FOR STOCK SPLITS
POLYGON_SPLIT_URL = 'https://api.polygon.io/v2/reference/splits/{}?apiKey={}'
#URL FOR TICKER TYPES
POLYGON_TYPES_URL = 'https://api.polygon.io/v2/reference/types?apiKey={}'

#%% 
# Get the list of all supported tickers from Polygon.io
def get_tickers(url = POLYGON_TICKERS_URL):
    page = 1

    session = requests.Session()
    # Initial request to get the ticker count
    r = session.get(POLYGON_TICKERS_URL.format(page, ALPACA_API_KEY))
    data = r.json()

    # This is to figure out how many pages to run pagination 
    count = data['count']
    print('total tickers ' + str(count))
    pages = math.ceil(count / data['perPage'])

    # Pull in all the pages of tickers
    for pages in range (2, pages+1):  # For production
        r = session.get(POLYGON_TICKERS_URL.format(page, ALPACA_API_KEY))
        data = r.json()
        df = pd.DataFrame(data['tickers'])
        df.to_csv('data/tickers/{}.csv'.format(page), index=False)
        print('Page {} processed'.format(page))
        page += 1
        
    return('Processes {} pages of tickers'.format(page-1))


# Stich all of these csv files into one dataframe for analysis
def combine_tickers(directory):

    df = pd.DataFrame()

    for f in os.listdir(directory):
        df2 = pd.read_csv('{}/{}'.format(directory, f))
        df = df.append(df2)
    
    # Read out a copy of the file to a csv for later analysis
    df.set_index('ticker', inplace=True)
    df.drop_duplicates()  # Just in case any tickers get pulled twice
    df.to_csv('polygon_tickers.csv')
    
    # Returns result df of filtered tickers
    return df


# Function to filter tickers by exchange
def filter_us_exch(ticker_df):
    
    # Keep only U.S. Dollar denominated securities
    df = ticker_df[(ticker_df.currency == 'USD') & (ticker_df.locale == 'US')]
    # Keep only the primary U.S. exchanges
    exch = ['AMX','ARCA','BATS','NASDAQ','NSC','NYE']
    df = df[df['primaryExch'].isin(exch)]
    # Filter out preferred stock, american depositry receipts, closed end funds, reit
    stockTypes = ['PFD','ADR','CEF','MLP','REIT','RIGHT','UNIT','WRT']
    df = df[df['type'].isin(stockTypes) == False]
    
    df.to_csv('polygon_tickers_us.csv')

    # Create a list of symbols to loop through
    symbols = df.index.tolist()

    # Returns list of filtered symbols
    return symbols


# Get the aggregated bars for the symbols
def get_bars(symbolslist, outdir, start, end):

    session = requests.Session()
    # In case I run into issues, retry my connection
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))
    count = 0
    
    barlog = open("barlog.txt", "w")
    
    for symbol in symbolslist:
        try:
            r = session.get(POLYGON_AGGS_URL.format(symbol, start, end, ALPACA_API_KEY))
            if r:
                data = r.json()
            
                # create a pandas dataframe from the information
                if data['queryCount'] > 1:
                    df = pd.DataFrame(data['results'])
                    df['date'] = pd.to_datetime(df['t'], unit='ms')
                    df['date'] =  df['date'].dt.date.astype(str)
                    df.set_index('date', inplace=True)
                    df['symbol'] = symbol
                    
                    # Drop irrelevant columns and rename relevant ones
                    df.drop(columns=['vw', 't', 'n'], inplace=True)
                    df.rename(columns={'v': 'volume', 'o': 'open', 'c': 'close', 'h': 'high', 'l': 'low'}, inplace=True)
                    
                    # Export Results
                    df.to_csv('{}/{}.csv'.format(outdir, symbol), index=True)
                    count += 1

                    # Logging the results, I could write a short method for this to reuse
                    msg = (symbol + ' file created with record count ' + str(data['queryCount']))
                    print(msg)
                    barlog.write(msg)
                    barlog.write("\n")

                # Returning Error for Symbols with missing Data
                else:
                    msg = ('No data for symbol ' + str(symbol))
                    print(msg)
                    barlog.write(msg)
                    barlog.write("\n")
                    
            # Returning Error for Symbols with faulty response        
            else:
                msg = ('No response for symbol ' + str(symbol))
                print(msg)
                barlog.write(msg)
                barlog.write("\n")
                
        # Raise exception for Symbol but continue           
        except:
            msg = ('****** exception raised for symbol ' + str(symbol))
            print(msg)
            barlog.write(msg)
            barlog.write("\n")
    
    # End Process and Log Files Exported
    barlog.close()
    return ('{} file were exported'.format(count))


# Define a function to pull in the splits data
def get_splits(symbolslist, outdir):

    session = requests.Session()
    # In case I run into issues, retry my connection
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))
    count = 0
    
    # Get the split data
    for symbol in symbolslist:
        try:
            r = session.get(POLYGON_SPLIT_URL.format(symbol, ALPACA_API_KEY))
            if r:
                data = r.json()
                if data['count'] > 0:
                    df = pd.DataFrame(data['results'])
                    # Renaming Columns
                    df.rename(columns={'exDate': 'date', 'declaredDate': 'splitDeclaredDate'}, inplace=True)
                    
                    # Dropping Irrelevant Columns
                    df.drop(columns=['paymentDate'], inplace=True)
                    
                    # Indexing by Date
                    df.set_index('date', inplace=True)
                    
                    # Exporting Results
                    df.to_csv('{}/{}.csv'.format(outdir, symbol), index=True)
                    
                    # Log of results
                    print('split file for ' + symbol + ' ' + str(data['count']))
                    count += 1
                # No data error
                else:
                    print('No data for symbol ' + str(symbol))
            # No response error
            else:
                print('No response for symbol ' + str(symbol))
        # Raise exception but continue           
        except:
            print('****** exception raised for symbol ' + str(symbol))
    
    # Return split results
    return ('{} file were exported'.format(count))


# Polygon API returned results with incorrect split results
# Fix erroneous splits from Polygon with a manually created correction file 
def fix_splits(splitpath):
    # Get the split corrections to overwrite
    correct_df = pd.read_csv('split_corrections.csv')
    # create a list of symbols to fix
    symbols = correct_df['ticker'].tolist()
    # remove duplicates
    symbols = list(dict.fromkeys(symbols))

    # for symbol in symbols:
    for symbol in symbols:
        
    # Checking if splits exist for each ticker, adjusting if they exist
        if os.path.isfile('{}/{}.csv'.format(splitpath, symbol)):
            # Getting Split Data
            df = pd.read_csv('{}/{}.csv'.format(splitpath, symbol))
            
            # Left joining Split Corrections DF on Incorrect Splits DF
            df = pd.merge(df, correct_df, how='left', left_on=['date', 'ticker'], right_on=['date', 'ticker'])
            
            # Iterate over DF rows
            for index, row in df.iterrows():
                # Adjust bad dates
                if not pd.isnull(row.date_adj):
                    df.loc[index, 'date'] = row.date_adj
                # Adjust bad ratios
                if not pd.isnull(row.ratio_adj):
                    df.loc[index, 'ratio'] = row.ratio_adj
                # Else keep the same
                else:
                    df.loc[index, 'ratio'] = row.ratio_x
            
            # Format the dataframe for export
            df = df[['date', 'ticker', 'ratio']]
            df.set_index('date', inplace=True)

            # Overwrite the file with this new file
            df.to_csv('{}/{}.csv'.format(splitpath, symbol))
            print('Split file for {} corrected'.format(symbol))
        
        # File not Found Error
        else:
            print('no file found')
    
    # Return Statement Indicating Successful Completion
    return ('Split file corrections complete')


# Define a function to pull in the splits data
def get_divs(symbolslist, outdir):

    session = requests.Session()
    count = 0
    
    # Get the split data
    for symbol in symbolslist:
        r = session.get(POLYGON_DIV_URL.format(symbol, ALPACA_API_KEY))
        data = r.json()
        if data['count'] > 0:
            df = pd.DataFrame(data['results'])
            # df.rename(columns={'paymentDate': 'date'}, inplace=True)
            df.rename(columns={'exDate': 'date', 'amount': 'dividend',
                               'paymentDate': 'divPaymentDate',
                               'recordDate': 'divRecordDate',
                               'declaredDate': 'divDeclaredDate'}, inplace=True)
            df.set_index('date', inplace=True)
            df = df.groupby(df.index).first()
            df.to_csv('{}/{}.csv'.format(outdir, symbol), index=True)
            
            print('div file for ' + symbol + ' ' + str(data['count']))
            count += 1
            
    return ('{} file were exported'.format(count))


# Combine bars, splits and dividend - Secondary main function
def combine_bars(barpath, splitpath, divpath):

    # Initialize Count
    count = 0
    
    # Count paths
    for f in os.listdir(barpath):
        
        symbol = f[:-4]
        print(symbol)
        
        # Get the bar data
        if os.path.isfile('{}/{}.csv'.format(barpath, symbol)):
            bars = pd.read_csv('{}/{}.csv'.format(barpath, symbol), index_col='date')
            
            # get any splits
            if os.path.isfile('{}/{}.csv'.format(splitpath, symbol)):
                splits = pd.read_csv('{}/{}.csv'.format(splitpath, symbol), index_col='date')
                splits.drop(columns=['ticker'], inplace=True)
                
                # left join splits on bars
                bars = bars.merge(splits, left_index=True, right_index=True, how='left')

            # Else if no splits, keep bars the same
            else:
                bars = bars
            
            # Get any dividend payments
            if os.path.isfile('{}/{}.csv'.format(divpath, symbol)):
                # Combine with symbols
                divs = pd.read_csv('{}/{}.csv'.format(divpath, symbol), index_col='date')
                # Drop redudant columns
                divs.drop(columns=['ticker'], inplace=True)
                # Left join dividends on bars
                bars = bars.merge(divs, left_index=True, right_index=True, how='left')
            
            # Else if no dividends, keep bars df the same
            else:
                
                bars = bars
                
            # Export bars with split data and dividend data 
            bars.to_csv('data/bars_adj/{}.csv'.format(symbol))
            count += 1
        
    return ('{} adjusted bar file were exported'.format(count))


# Finally, adjusting the OHLCV data for stock splits - Main Function
def adj_bars(directory):

    # Initialize Count
    count = 0
    
    # Iterate over directory files
    for f in os.listdir(directory):

        # Get df
        df = pd.read_csv('{}/{}'.format(directory, f), index_col='date')
        
        # Adjust the split ratio if incorrect
        if 'ratio' in df.columns:
            df['ratio_adj'] = df['ratio']
            
        # Else, keep the ratio if correct
        else:
             df['ratio_adj'] = 1

        # Create a split factor, shifted to the day earlier.  Also, fill in any missing factors with 1
        df['split_factor'] = (1 / df['ratio_adj'].shift(-1)).fillna(1)
        #  Create a cumulative product of the splits, in reverse order using the []::-1]
        df['split_factor'] = df['split_factor'][::-1].cumprod()

        # Adjust the various OHLCV metrics
        df['volume_adj'] = df['volume'] * df['split_factor']
        df['open_adj'] = df['open'] / df['split_factor']
        df['close_adj'] = df['close'] / df['split_factor']
        df['high_adj'] = df['high'] / df['split_factor']
        df['low_adj'] = df['low'] / df['split_factor']
        df['dollar_volume'] = df['volume'] * df['close']

        # Export results
        df.to_csv('{}/{}'.format(directory, f))
        
        # Update the count
        count += 1
        
    return ('{} files was adjusted'.format(count))



# Get all the tickers on Polygon.io and save them to a data directory
get_tickers()

# Combine all the paginated ticker files together into one dataframe
symbols = combine_tickers('data/tickers')

# Filter down to the tickers I'm interestead in (this could also be done by modifying get_tickers)
symbols = filter_us_exch(symbols)

# Get all the aggregated bar/pricing data for each symbol in the filtered list
get_bars(symbols, 'data/bars', START_DATE, END_DATE)

#  Pull in all the stock splits
get_splits(symbols, 'data/splits')
# Fix data for about 50 splits from a correction file created manually
fix_splits('data/splits')

#  Pull in all the dividend data
get_divs(symbols, 'data/divs')

# Combine the bars (pricing data) with any splits and dividend payments
combine_bars('data/bars', 'data/splits', 'data/divs')

#  Create new and stock split adjusted OHLCV fields
adj_bars('data/bars_adj')


# Testing on AAPL 
bars = pd.read_csv('data/bars_adj/AAPL.csv')
bars['close'].plot()
